# Task B: Solve Dinner Planning with CSP

## Problem Description

"The Dupont family will visit us tonight," Mr. Martin announces. "The whole family, that is, Mr. and
Mrs. Dupont and their three children Emma, Georg and Ivana?" Mrs. Martin asks in dismay.
Mr. Martin replies: "That's the problem: they wanted to make a secret about who exactly wants to come. 
But we know: If Mr. Dupont comes, then his wife will come, too.
At least one of the two children Ivana and Georg will come. Either Mrs. Dupont or Emma will come.
Either Emma and Georg come both, or they both don't come. And if Ivana comes, then also Georg and
Mr. Dupont will come."

Help the Martins with the dinner planning: Who will visit the Martins tonight?

## Abbreviations

Mr H
Mrs W
Emma E
Georg G
Ivana I

-> H W E G I

## Constrains

H -> W
I or G
W xor E
E <-> G
I -> (G and H)

## Constrains in CNF

(-H or W)
(I or G)
(W or E) and (not W or not E)
(-E or G) and (-G or E)
(-I or G) and (-I or H)

## Constrains in CNF under each other

(-H or W)        -> ["-H", "W"]
(I or G)         -> ["I", "G"]
(W or E)         -> ["W", "E"]
(not W or not E) -> ["-W", "-E"]
(-E or G)        -> ["-E", "G"]
(-G or E)        -> ["-G", "E"]
(-I or G)        -> ["-I", "G"]
(-I or H)        -> ["-I", "H"]



In [ ]:
# variables: e. g.: ["A", "B", "C", "D"]
# or_constrains: e. g.: [["A", "B"], ["-B", "C"], ["C", "-D"]]
# combination: e. g.: ["A", "B", "C", "D"] or ["A", "-B", "C", "-D"] or ["-A", "-B", "-C", "-D"]


def is_or_constrain_always_satisfied(or_constrain: list[str]):
    for var in or_constrain:
        if "-" + var in or_constrain:
            return True
    return False

def is_or_constrain_satisfied(or_constrain: list[str], combination: list[str]):

    if is_or_constrain_always_satisfied(or_constrain):
        return True

    for var in or_constrain:
        if var in combination:
            return True
    return False
        

assert(False == is_or_constrain_always_satisfied(["A", "B", "-C"]))
assert(True == is_or_constrain_always_satisfied(["A", "B", "-A"]))
assert(True == is_or_constrain_satisfied(
    or_constrain=["A", "B", "-C"],
    combination=["A"]
))
assert(True == is_or_constrain_satisfied(
    or_constrain=["A", "-A"],
    combination=["B"]
))
assert(True == is_or_constrain_satisfied(
    or_constrain=["A", "-B", "C"],
    combination=["C", "-B"]
))
assert(True == is_or_constrain_satisfied(
    or_constrain=["A", "-B", "C"],
    combination=["C", "B"]
))
assert(False == is_or_constrain_satisfied(
    or_constrain=["A", "-B", "C"],
    combination=["-C", "B"]
))
assert(False == is_or_constrain_satisfied(
    or_constrain=["A", "-B", "C"],
    combination=[]
))

In [ ]:
def simple_bin_csp_solver(variables: list[str], or_constrains: list[list[str]]):

    # first with brute force

    # generate all combinations
    combinations = []
    for i in range(2**len(variables)):
        combinations.append([])
        for j in range(len(variables)):
            combinations[i].append(variables[j] if i & (1 << j) else "-" + variables[j])

    # go through all combinations and return all that satisfy all or constrains
    solutions = []
    for combination in combinations:
        is_solution = True
        for or_constrain in or_constrains:
            if not is_or_constrain_satisfied(or_constrain, combination):
                is_solution = False
                break
        if is_solution:
            solutions.append(combination)

    return solutions

In [158]:

import copy

def neg(var: str):
    if var[0] == "-":
        return var[1:]
    else:
        return "-" + var
    
def is_neg(var: str):
    return var[0] == "-"
    
assert("A" == neg("-A"))
assert("-A" == neg("A"))
assert(True == is_neg("-A"))
assert(False == is_neg("A"))

def unique(l: list[str]):
    result = list(set(l))
    result.sort()
    return result

def sort_array(l: list[str]):
    result = copy.deepcopy(l)
    result.sort()
    return result

assert(["A", "B", "C"] == unique(["A", "B", "C", "A", "B", "C"]))

In [160]:
def remove_unit_clauses(result, clauses):
    while True:
        unitClauses = []
        oldClauses = copy.deepcopy(clauses)
        for clause in clauses:
            if len(clause) == 1 and clause[0] not in unitClauses:
                unitClauses.append(clause[0])
        if len(unitClauses) == 0:
            break

        # remove all clauses that contain unit clauses (literals)
        for clause in oldClauses:
            for unitClause in unitClauses:
                if unitClause in clause:
                    try:
                        clauses.remove(clause)
                    except:
                        pass

        # remove all negated unit clauses (literals) from all clauses
        for clause in clauses:
            for unitClause in unitClauses:
                if neg(unitClause) in clause:
                    clause.remove(neg(unitClause))

        # add all unit clauses to the result
        for unitClause in unitClauses:

            # if unitClause is already negated in the result, then there is no solution -> throw error
            if neg(unitClause) in result:
                return [], clauses
                # raise Exception("no solution")
            
            # if unitClause is not already in the result, then add it
            if unitClause not in result:
                result.append(unitClause)

    return unique(result), clauses

# test this
result = []
clauses = [
    ["-W"],
    ["-H", "W"],
    ["I", "G"],
    ["W", "E"],
    ["-W", "-E"],
    ["-E", "G"],
    ["-G", "E"],
    ["-I", "G"],
    ["-I", "H"],
]

result, clauses = remove_unit_clauses(result, clauses)
print("result: ", result)
print("clauses: ", clauses)

result:  ['-H', '-I', '-W', 'E', 'G']
clauses:  []


In [168]:
def simple_bin_csp_solver(result: list[str], clauses: list[list[str]]):

    # first remove all unit clauses
    result, clauses = remove_unit_clauses(result, clauses)

    # if null clause is present, return an empty list
    for clause in clauses:
        if len(clause) == 0:
            return []
        
    # if no clauses are left, return the result
    if len(clauses) == 0:
        return result
    
    # get a literal and build two new clauses
    literal = clauses[0][0]
    clauses_1 = copy.deepcopy(clauses) + [[literal]]
    clauses_2 = copy.deepcopy(clauses) + [[neg(literal)]]

    # call recursively with the two new clauses
    result_1 = simple_bin_csp_solver(copy.deepcopy(result), clauses_1)
    if len(result_1) > 0:
        result_with_literal = [literal] + result_1
        return unique(result_with_literal)
    
    result_2 = simple_bin_csp_solver(copy.deepcopy(result), clauses_2)
    if len(result_2) > 0:
        result_with_neg_literal = [neg(literal)] + result_2
        return unique(result_with_neg_literal)
    
    return []

# solve the dinner planning problem
constrains = [
    ["-H", "W"],
    ["I", "G"],
    ["W", "E"],
    ["-W", "-E"],
    ["-E", "G"],
    ["-G", "E"],
    ["-I", "G"],
    ["-I", "H"],
]

should_solution = ["-H", "-W", "-I", "G", "E"]
is_solution = simple_bin_csp_solver([], constrains)
print(is_solution)
assert(sort_array(should_solution) == sort_array(is_solution))

['-H', '-I', '-W', 'E', 'G']
